## Обработка SKU

In [18]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

name = input('Введите имя файла (отчёт): ')
name = name + '.xlsx'
df = pd.read_excel(name, sheet_name='Данные')

Введите имя файла (отчёт): Отчет_PMI_Dina_Anvar_12_Февраль


In [19]:
def add_missing_names(before, after):
    if isinstance(before, float) or isinstance(after, float):
        if before != before:
            return after
        return before
    before_set = set(before.split("; "))
    after_set = set(after.split("; "))
    missing = after_set - before_set
    if missing:
        before = before + "; " + "; ".join(missing)
    if isinstance(before, str):
        before = before.replace("nan", "")
    return before.rstrip("; ")

def compare_columns(df, before_cols, after_cols):
    result_cols = []
    for i in range(0, len(before_cols)):
        before_col = before_cols[i]
        after_col = after_cols[i]
        result_col = []
        for j in range(len(df[before_col])-1):
            before = df[before_col][j]
            after = df[after_col][j+1]
            if pd.isna(before):
                result_col.append(after)
            else:
                before_set = set(str(before).split("; "))
                after_set = set(str(after).split("; "))
                missing = after_set - before_set
                if missing:
                    before = str(before) + "; " + "; ".join(missing)
                result_col.append(before)
        result_col = [str(i).replace('nan', '') for i in result_col]
        result_col = [i.rstrip("; ") for i in result_col]
        result_cols.append((before_col, result_col))
    return result_cols


# Load the existing table into a Pandas DataFrame
#df = pd.read_excel('Отчет_PMI_Magnum_13_Февраль.xlsx', sheet_name='Данные')

before_cols = [col for col in df.columns if "Отметьте отсутствие позиции до работы _Касса " in col]

# Get the names of the after columns
after_cols = [col for col in df.columns if "Отметьте отсутствие позиции после работы _Касса " in col]

# Call the compare_columns function and store the result in a list of tuples
result = compare_columns(df, before_cols, after_cols)

# Create a new empty DataFrame with the same number of rows as the original DataFrame
result_df = pd.DataFrame(index=df.index)

# Loop through the results for each "before" column and assign the corresponding "after" column to the result DataFrame
for before_col, result_col in result:
    after_col = before_col.replace('Отметьте отсутствие позиции до работы', 'Отметьте отсутствие позиции после работы')
    result_df[before_col] = result_col + [None]

df.update(result_df)

# write data to Excel without hyperlinks
with pd.ExcelWriter('new_'+name, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Данные', index=False)
    workbook = writer.book
    worksheet = writer.sheets['Данные']
    worksheet.column_dimensions['A'].width = 30
        
# Save the result to a new Excel
#df.to_excel('new_Отчет_PMI_Magnum_13_Февраль.xlsx.xlsx')
print('\n'+'Файл сохранён под именем: new_format_' + name)


Файл сохранён под именем: new_format_Отчет_PMI_Dina_Anvar_12_Февраль.xlsx


Если вышла ошибка, то нужно выполнить следующую ячейку (это только для первого раза)

In [5]:
pip install XlsxWriter

Note: you may need to restart the kernel to use updated packages.
